# Data: The NBA drafting, player, and game data are sourced from stats.nba.com via the nba_api.
# exploratory data analysis of NBA historic drafting, player, and team data.

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.express as px
import os

In [36]:
# connect to SQL database
db_path = r'C:\Users\Administrator\Desktop\basketball.sqlite'
connection = sql.connect(db_path) 
table = pd.read_sql_query("SELECT * FROM sqlite_master ", connection)
print(table)

     type                   name               tbl_name  rootpage  \
0   table                 Player                 Player         2   
1   table                   Team                   Team         4   
2   table        Team_Attributes        Team_Attributes         5   
3   table           Team_History           Team_History         6   
4   table      Player_Attributes      Player_Attributes        15   
5   table         Game_Officials         Game_Officials      8375   
6   table  Game_Inactive_Players  Game_Inactive_Players      8997   
7   table            Team_Salary            Team_Salary     13415   
8   table          Player_Salary          Player_Salary     23558   
9   table                  Draft                  Draft     23559   
10  table          Draft_Combine          Draft_Combine     23562   
11  table          Player_Photos          Player_Photos     23565   
12  table            Player_Bios            Player_Bios     22541   
13  table                   Game  

In [243]:
table1 = pd.read_sql("SELECT * from Game_Inactive_Players ", connection)
print(table1)
list(table1.columns.values)

      PLAYER_ID FIRST_NAME   LAST_NAME JERSEY_NUM     TEAM_ID    TEAM_CITY  \
0           184      Bobby      Phills         14  1610612739    Cleveland   
1           781       Will      Perdue         41  1610612759  San Antonio   
2           120     Steven       Smith          8  1610612737      Atlanta   
3           698      Eddie     Johnson          8  1610612754      Indiana   
4            32   Clifford      Rozier         44  1610612753      Orlando   
...         ...        ...         ...        ...         ...          ...   
98674   1630169     Tyrese  Haliburton       0     1610612758   Sacramento   
98675   1626158    Richaun      Holmes       22    1610612758   Sacramento   
98676   1628368   De'Aaron         Fox       5     1610612758   Sacramento   
98677    203084   Harrison      Barnes       40    1610612758   Sacramento   
98678   1628963     Marvin  Bagley III       35    1610612758   Sacramento   

       TEAM_NAME TEAM_ABBREVIATION     GAME_ID  
0      Cavalie

['PLAYER_ID',
 'FIRST_NAME',
 'LAST_NAME',
 'JERSEY_NUM',
 'TEAM_ID',
 'TEAM_CITY',
 'TEAM_NAME',
 'TEAM_ABBREVIATION',
 'GAME_ID']

### 1. How does the NBA drafting count change over time from 1949 to 2020?

In [34]:
## How does the NBA drafting count change over time from 1949 to 2020?

query =""" select yeardraft as year,count(nameplayer) as total_drafted 
            from draft
            group by yeardraft"""

draftyear = pd.read_sql(query,connection)
print(draftyear)
fig = px.line(draftyear,x='year',y='total_drafted',
             title='NBA Drafting Trend from 1949 to 2020')
fig.show()

      year  total_drafted
0   1949.0             75
1   1950.0            121
2   1951.0             87
3   1952.0            106
4   1953.0            122
..     ...            ...
67  2016.0             60
68  2017.0             60
69  2018.0             60
70  2019.0             60
71  2020.0             60

[72 rows x 2 columns]


#### It's quite surprising that NBA has way more drafts in the 70s and 80s comparing the 2000s.
#### In the year of 1970, there were 239 drafts and there were 228 drafts in the year of 1984 whereas NBA only
#### drafted around 60 players per year in the recent years.

### 2. What are the top 10 NBA team that drafted most number of player from university?

In [197]:
## What are the top 10 NBA team that drafted most number of player from university?

query = ''' select nameTeam , count(distinct nameplayer) as count_drafted
            from draft 
            group by idteam
            order by count_drafted desc
            limit 10'''
teamdrafted = pd.read_sql(query,connection)
print(teamdrafted)
fig1 = px.bar(teamdrafted,x='nameTeam',y='count_drafted',
                title = 'Top 10 NBA team that drafted most number of player from university')
fig1.show()

                nameTeam  count_drafted
0          Atlanta Hawks            513
1       Sacramento Kings            510
2     Philadelphia 76ers            487
3        New York Knicks            471
4  Golden State Warriors            450
5         Boston Celtics            448
6        Detroit Pistons            430
7     Los Angeles Lakers            426
8     Washington Wizards            411
9          Chicago Bulls            341


#### Atlanta Hawks (514), Sacramento Kings (511), Philadelphia 76ers (487) are the top 3 teams that drafted the most number of players over time.

### 3. When did the teams first start to draft players from the universities?

In [208]:
## When did the teams first start to draft players from the universities?

query = """select min(yeardraft) as first_year ,
            nameteam as team,
            COUNT(DISTINCT yearDraft) AS years_drafted,
            COUNT(DISTINCT idPlayer) AS total_drafted_to_date
            from draft
            group by idteam
            order by first_year"""
team_first_draft_year = pd.read_sql(query,connection)
print(team_first_draft_year)

    first_year                       team  years_drafted  \
0       1949.0          Baltimore Bullets              6   
1       1949.0              Chicago Stags              2   
2       1949.0     Indianapolis Olympians              4   
3       1949.0    Providence Steamrollers              1   
4       1949.0          St. Louis Bombers              1   
5       1949.0        Washington Capitols              2   
6       1949.0             Boston Celtics             72   
7       1949.0      Philadelphia Warriors             69   
8       1949.0         Minneapolis Lakers             71   
9       1949.0            New York Knicks             69   
10      1949.0           Rochester Royals             71   
11      1949.0  Ft. Wayne Zollner Pistons             72   
12      1950.0      Tri-Cities Blackhawks             70   
13      1950.0         Syracuse Nationals             71   
14      1961.0            Chicago Packers             58   
15      1966.0              Chicago Bull

In [204]:
print("For each year, how many team first started drafting new players?")
team_first_draft_year["first_year"].value_counts()

For each year, how many team first started drafting new players?


1949.0    12
1977.0     4
1970.0     3
1950.0     2
1967.0     2
1968.0     2
1988.0     2
1989.0     2
1995.0     2
1961.0     1
1966.0     1
1974.0     1
1980.0     1
2003.0     1
Name: first_year, dtype: int64

#### From the above sumary table, we can observe that most of the teams first started drafting players in 1949-1950 season.

### 4. Where are the players coming from?

In [209]:
### Where are the players coming from? 
### Do most of the players coming from high school, university, or from other professional basketball team?

query ="""select typeOrganizationFrom as draft_from, count(distinct idplayer) as count_player
        from draft
        group by draft_from"""
draft_player_from = pd.read_sql(query,connection)
print(draft_player_from)

px.pie(draft_player_from,values='count_player' ,names='draft_from')


           draft_from  count_player
0                None            15
1  College/University          7389
2         High School            48
3     Other Team/Club           304


#### Historically speaking, we can see that the majority of the players (95.3%) were been drafted from either College or University.
#### Very small fraction of the players are drafted from either other baskbetball team/club or high school to play in NBA.

### ️5. From 1949 to 2020, how did the total number of participated team count changes 
### and how did the total game count changes along with it overall?

In [72]:
from plotly.subplots import make_subplots # make subplot for dual axes plot

In [218]:
### ️From 1949 to 2020, how did the total number of participated team count changes 
### and how did the total game count changes along with it overall?
query = """ select id , nickname, min(YEARFOUNDED) as yearfound from team_history
            group by id
            order by yearfound """
query =""" select season_id-20000 as season,count(distinct TEAM_NAME_HOME) as count_team,count(distinct GAME_ID) as count_game
            from game
            group by season
            order by season """

participated_team = pd.read_sql(query,connection)
print(participated_team)

fig = make_subplots(specs=[[{"secondary_y": True}]])
total_count_team = px.bar(participated_team, x='season',y='count_team')
total_count_game = px.line(participated_team, x='season',y='count_game')
total_count_team.update_traces(name="team totals", showlegend =True)
total_count_game.update_traces(name='game totals', showlegend =True,line_color='red')
fig.add_trace(total_count_team.data[0], secondary_y=False)
fig.add_trace(total_count_game.data[0], secondary_y=True)
fig.update_yaxes(title_text='team totals',secondary_y=False)
fig.update_yaxes(title_text='game totals',secondary_y=True)

    season  count_team  count_game
0     1946          11         331
1     1947           8         194
2     1948          12         360
3     1949          17         563
4     1950          11         354
..     ...         ...         ...
70    2016          30        1230
71    2017          30        1230
72    2018          30        1230
73    2019          30        1059
74    2020          30        1080

[75 rows x 3 columns]


#### From 1946 to 2020, we can observe that the total game and total team counts increases over the years 0 from 11 teams playing 331 games to 30 teams playing 1080 games in the 2020 season.
#### 2020-2021 is special: Each team will play 72 regular-season games, which is 10 games fewer than in a typical, 82-game NBA season.

### 6. From 1949 to 2020, how did home game game won percentage among all the games change over time?

In [216]:
### From 1949 to 2020, how did home game game won percentage among all the games change over time?

query = '''
        select (game1.season_id-20000) as season, round(100 * count(game1.wl_home)/game2.total_game,3) as homewin_percentage
        from game as game1
        join (select game_id,season_id, count(wl_home) as total_game from game group by season_id) as game2
        on game1.season_id = game2.season_ID
        where game1.wl_home ='W'
        group by game1.season_id
          '''



fig = pd.read_sql(query,connection)
print(fig)
px.line(fig,x='season',y='homewin_percentage')

    season  homewin_percentage
0     1946                61.0
1     1947                57.0
2     1948                63.0
3     1949                69.0
4     1950                74.0
..     ...                 ...
70    2016                58.0
71    2017                57.0
72    2018                59.0
73    2019                55.0
74    2020                53.0

[75 rows x 2 columns]


#### Overall, the home game won percentage decreased from 75.35% to 51.67% from 1950s to 2020.

In [219]:
query = """
    SELECT 
        SEASON_ID-20000 AS season,
        TEAM_ID_HOME AS team_id,
        TEAM_NAME_HOME AS team_name,
        SUM(CASE WL_HOME 
                WHEN'W' THEN 1
                ELSE 0
            END) AS win_count,
        COUNT(TEAM_ID_HOME) AS team_game_count,
        "home" AS game_location
    FROM Game
    GROUP BY SEASON_ID, TEAM_ID_HOME 
    
    UNION
    
    SELECT 
        SEASON_ID-20000 AS season,
        TEAM_ID_AWAY AS team_id,
        TEAM_NAME_AWAY AS team_name,
        SUM(CASE WL_AWAY 
                WHEN'W' THEN 1
                ELSE 0
            END) AS win_count,
        COUNT(TEAM_ID_AWAY) AS team_game_count,
        "away" AS game_location
    FROM Game
    GROUP BY SEASON_ID, TEAM_ID_AWAY
"""
team_level_home_game_stats = pd.read_sql(query, connection)
team_level_home_game_stats["win_percentage"] = round(
    100 * team_level_home_game_stats["win_count"] / team_level_home_game_stats["team_game_count"], 2)
team_level_home_game_stats

,season,team_id,team_name,win_count,team_game_count,game_location,win_percentage
0,1946,1610610025,Chicago Stags,17,30,away,56.67
1,1946,1610610025,Chicago Stags,22,31,home,70.97
2,1946,1610610026,Cleveland Rebels,13,30,away,43.33
3,1946,1610610026,Cleveland Rebels,17,30,home,56.67
4,1946,1610610028,Detroit Falcons,8,30,away,26.67
...,...,...,...,...,...,...,...
3141,2020,1610612764,Washington Wizards,19,36,home,52.78
3142,2020,1610612765,Detroit Pistons,7,37,away,18.92
3143,2020,1610612765,Detroit Pistons,13,36,home,36.11
3144,2020,1610612766,Charlotte Hornets,15,36,away,41.67


In [221]:
cols_to_drop = ["team_id", "team_name", "win_count", "team_game_count"]
median_win_pct = team_level_home_game_stats.drop(cols_to_drop, axis=1).groupby(["season", "game_location"]).mean().reset_index()
print(median_win_pct)

px.line(median_win_pct, x="season", y="win_percentage", 
           color="game_location", title="1946-2020: median of the team level game won percentage by game location")

     season game_location  win_percentage
0      1946          away       38.320000
1      1946          home       61.464545
2      1947          away       43.075000
3      1947          home       56.283750
4      1948          away       36.774167
..      ...           ...             ...
145    2018          home       59.268667
146    2019          away       44.347333
147    2019          home       54.787333
148    2020          away       44.877667
149    2020          home       51.407000

[150 rows x 3 columns]


#### With some variation, we can observe that the home game game won percentages have been relatively higher than the away game game won percentage. 
#### The areas between the two curves have decreased over the years.

### 7. How does the free throw percentage (FT%) changes over time from 1949 to 2020? 
### For each season, which team had the best FT%?

In [229]:
### How does the free throw percentage (FT%) changes over time from 1949 to 2020? 
### For each season, which team had the best FT%?

query = '''    SELECT 
        SEASON_ID-20000 AS season,
        TEAM_ID_HOME AS team_id,
        TEAM_NAME_HOME AS team_name,
        FT_PCT_HOME AS free_throw_percentage,
        COUNT(TEAM_ID_HOME) AS team_game_count,
        "home" AS game_location
    FROM Game
    GROUP BY SEASON_ID, TEAM_ID_HOME 
    
    UNION
    
    SELECT 
        SEASON_ID-20000 AS season,
        TEAM_ID_AWAY AS team_id,
        TEAM_NAME_AWAY AS team_name,
        FT_PCT_AWAY AS free_throw_percentage,
        COUNT(TEAM_ID_AWAY) AS team_game_count,
        "away" AS game_location
    FROM Game
    GROUP BY SEASON_ID, TEAM_ID_AWAY'''
FT_PCT=pd.read_sql(query,connection)
print(FT_PCT)

FT_PCT=FT_PCT.dropna().query("free_throw_percentage < 1").reset_index(drop=True)
cols_to_drop = ["team_id", "team_name", "team_game_count"]
median_ft_pct = FT_PCT.drop(cols_to_drop, axis=1).groupby(["season", "game_location"]).mean().reset_index()
px.line(median_ft_pct, x="season", y="free_throw_percentage", 
           color="game_location", title="1946-2020: median of the team level free throw percentage by game location")

      season     team_id           team_name  free_throw_percentage  \
0       1946  1610610025       Chicago Stags                    NaN   
1       1946  1610610025       Chicago Stags                    NaN   
2       1946  1610610026    Cleveland Rebels                    NaN   
3       1946  1610610026    Cleveland Rebels                  0.667   
4       1946  1610610028     Detroit Falcons                    NaN   
...      ...         ...                 ...                    ...   
3141    2020  1610612764  Washington Wizards                  0.696   
3142    2020  1610612765     Detroit Pistons                  0.375   
3143    2020  1610612765     Detroit Pistons                  0.719   
3144    2020  1610612766   Charlotte Hornets                  0.533   
3145    2020  1610612766   Charlotte Hornets                  0.625   

      team_game_count game_location  
0                  30          away  
1                  31          home  
2                  30          ho

#### The FP% starts from 70% and increased over the years to 75% for median NBA season overall FP% among all the games.

### 8. How does the three point field goal percentage (3P%) changes over time from 1949 to 2020?
### For each season, which team had the best 3P%?

In [228]:
### How does the three point field goal percentage (3P%) changes over time from 1949 to 2020?
### For each season, which team had the best 3P%?
query = '''select SEASON_ID-20000 AS SEASON,TEAM_ID_HOME AS TEAM_ID,TEAM_NAME_HOME AS TEAM_NAME, FG3_PCT_HOME AS FG3_PCT, 
        "HOME" AS LOCATION FROM GAME
        GROUP BY SEASON,TEAM_ID_HOME 
        UNION
        SELECT SEASON_ID-20000 AS SEASON,TEAM_ID_AWAY AS TEAM_ID,TEAM_NAME_AWAY AS TEAM_NAME, FG3_PCT_AWAY AS FG3_PCT, 
        'AWAY' AS LOCATION FROM GAME
        GROUP BY SEASON,TEAM_ID_AWAY'''
FG3_PCT = pd.read_sql(query,connection)
print(FG3_PCT)

FG3_PCT = FG3_PCT.dropna().query("FG3_PCT < 1").reset_index(drop=True)
cols_to_drop = ["TEAM_ID", "TEAM_NAME"]
FG3_PCT_MEDIAN=FG3_PCT.drop(cols_to_drop, axis=1).groupby(["SEASON", "LOCATION"]).mean().reset_index()
print(FG3_PCT_MEDIAN)
px.line(FG3_PCT_MEDIAN.query("SEASON > 1986"),y='FG3_PCT',x='SEASON',color='LOCATION')

      SEASON     TEAM_ID           TEAM_NAME  FG3_PCT LOCATION
0       1946  1610610025       Chicago Stags      NaN     AWAY
1       1946  1610610025       Chicago Stags      NaN     HOME
2       1946  1610610026    Cleveland Rebels      NaN     AWAY
3       1946  1610610026    Cleveland Rebels      NaN     HOME
4       1946  1610610028     Detroit Falcons      NaN     AWAY
...      ...         ...                 ...      ...      ...
3141    2020  1610612764  Washington Wizards    0.481     AWAY
3142    2020  1610612765     Detroit Pistons    0.229     AWAY
3143    2020  1610612765     Detroit Pistons    0.356     HOME
3144    2020  1610612766   Charlotte Hornets    0.364     AWAY
3145    2020  1610612766   Charlotte Hornets    0.406     HOME

[3146 rows x 5 columns]
    SEASON LOCATION   FG3_PCT
0     1981     HOME  0.250000
1     1982     AWAY  0.000000
2     1982     HOME  0.000000
3     1983     AWAY  0.000000
4     1983     HOME  0.400000
..     ...      ...       ...
74    201

### 9. Who are the top 10 NBA players based on 2020-2021 season salary?

In [232]:
### Who are the top 10 NBA players based on 2020-2021 season salary?

query = '''select slugSeason as season,
            nameTeam,namePlayer,
            2021 - strftime('%Y', Player_Attributes.BIRTHDATE) AS age,
            ROUND(value/1000000,1) AS salary_in_millions 
        from Player_Salary
        JOIN Player ON
            Player_Salary.namePlayer = Player.full_name
        JOIN Player_Attributes ON
            Player.ID = Player_Attributes.ID
            where slugseason = '2020-21' 
            order by value desc
            limit 10
            '''
top10salary = pd.read_sql(query,connection)
top10salary

,season,nameTeam,namePlayer,age,salary_in_millions
0,2020-21,Golden State Warriors,Stephen Curry,33,43.0
1,2020-21,Phoenix Suns,Chris Paul,36,41.4
2,2020-21,Washington Wizards,Russell Westbrook,33,41.4
3,2020-21,Brooklyn Nets,James Harden,32,41.3
4,2020-21,Houston Rockets,John Wall,31,41.3
5,2020-21,Brooklyn Nets,Kevin Durant,33,40.1
6,2020-21,Los Angeles Lakers,LeBron James,37,39.2
7,2020-21,Los Angeles Clippers,Paul George,31,35.5
8,2020-21,Golden State Warriors,Klay Thompson,31,35.4
9,2020-21,Utah Jazz,Mike Conley,34,34.5


#### From the above table results, we can see that Stephen Curry from Golden State Warriors have 2020-21 season salary of 43 Millions.
#### The top 10 earning NBA players are all in their 30s.

## 10.What are the game play stats for the top 10 earning NBA players based on the data available in this table?

In [233]:
## What are the game play stats for the top 10 earning NBA players based on the data available in this table?

query = '''SELECT namePlayer AS player_name,
        nameTeam AS team_name,
        Player_Attributes.DRAFT_YEAR AS draft_year,
        Player_Attributes.POSITION AS game_position,
        Player_Attributes.PTS AS points,
        Player_Attributes.AST AS assists,
        Player_Attributes.REB AS rebounds,
        ROUND(value/1000000) AS salary_in_millions
    FROM Player_Salary
    JOIN Player ON
        Player_Salary.namePlayer = Player.full_name
    JOIN Player_Attributes ON
        Player.ID = Player_Attributes.ID
    WHERE slugSeason = '2020-21'
    ORDER BY salary_in_millions DESC
    LIMIT 10 '''

player_salary_top_10_game_stats = pd.read_sql(query, connection)
player_salary_top_10_game_stats

,player_name,team_name,draft_year,game_position,points,assists,rebounds,salary_in_millions
0,Stephen Curry,Golden State Warriors,2009,Guard,29.0,6.2,5.3,43.0
1,James Harden,Brooklyn Nets,2009,Guard,25.3,11.2,7.9,41.0
2,John Wall,Houston Rockets,2010,Guard,21.1,6.3,3.5,41.0
3,Chris Paul,Phoenix Suns,2005,Guard,15.7,8.8,4.7,41.0
4,Russell Westbrook,Washington Wizards,2008,Guard,21.7,10.3,9.5,41.0
5,Kevin Durant,Brooklyn Nets,2007,Forward,29.0,5.3,7.3,40.0
6,LeBron James,Los Angeles Lakers,2003,Forward,25.4,7.9,7.9,39.0
7,Klay Thompson,Golden State Warriors,2011,Guard,21.5,2.4,3.8,35.0
8,Paul George,Los Angeles Clippers,2010,Forward,22.5,5.5,6.0,35.0
9,Mike Conley,Utah Jazz,2007,Guard,16.4,5.5,3.4,35.0
